In [377]:
import numpy as np
from numpy import linalg as LA
from numpy.linalg import inv
import math

import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import cm

from sklearn import metrics
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

# Load data data
ground_truth_poses = np.loadtxt("../data/experiments/pose_scanner_leica_affine.txt") # id,x,y,z,w,x,y,z
ground_truth_poses = ground_truth_poses[:30, :]
ground_truth_poses = np.array([ground_truth_poses]).T        

corrected_poses = np.loadtxt("../data/experiments/corrected_poses.txt")              # idA,idB,x,y,z,w,x,y,z
corrected_poses = np.array([corrected_poses]).T

results = np.loadtxt("../data/experiments/compare_results.txt")                     # idA,idB,
results = np.array([results]).T                                                     # fov_overlap, octree_overlap
                                                                                    # alignability, alignment_risk
                                                                                    # degeneracy, ICN
# ground_truth_poses
# corrected_poses[2:]

In [378]:
# Compute translation error
def translationError(xg, yg, zg, xc, yc, zc):
   
    trans_err = np.zeros((xc.shape[0],1))
    gt_index = 0
    for i in range(0, xc.shape[0]):
        if (gt_index > xg.shape[0]-1):
            gt_index = 0
  
        trans_err[i] =  math.sqrt( pow((xg[gt_index] - xc[i]),2.0) + 
                                   pow((yg[gt_index] - yc[i]),2.0) +
                                   pow((zg[gt_index] - zc[i]),2.0) );
        gt_index = gt_index+1
    return trans_err

In [379]:
transl_err = translationError(ground_truth_poses[1], # [col][row]
                              ground_truth_poses[2],
                              ground_truth_poses[3],
                              corrected_poses[2],
                              corrected_poses[3],
                              corrected_poses[4])

# ground_truth_poses[2][1]
# corrected_poses[2][29]
# transl_err

In [380]:
# Compute rotation error
def quaternion_matrix(quaternion):
    quaternion = quaternion / LA.norm(quaternion)
#     print quaternion
    w = quaternion[0]
    x = quaternion[1]
    y = quaternion[2]
    z = quaternion[3]
    
    tmp = np.array([[1 - 2*y*y - 2*z*z,   2*x*y - 2*z*w,       2*x*z + 2*y*w    ],
                    [2*x*y + 2*z*w,       1 - 2*x*x - 2*z*z,   2*y*z - 2*x*w    ],
                    [2*x*z - 2*y*w,       2*y*z + 2*x*w,       1 - 2*x*x - 2*y*y]])
    result = tmp[:, :, 0]
    return result

def rotationError3D(q1g, q2g, q3g, q4g, q1c, q2c, q3c, q4c):
    rot_err = np.zeros((q1c.shape[0],1))
#     print rot_err.shape
    gt_index = 0
    for i in range(0, q1c.shape[0]):
        if (gt_index > q1g.shape[0]-1):
            gt_index = 0
            
#         print i, gt_index
        rot_g = quaternion_matrix([q1g[gt_index], q2g[gt_index], q3g[gt_index], q4g[gt_index]])
#         print rot_g
#         print q1g[gt_index]
        rot_c = quaternion_matrix([q1c[i], q2c[i], q3c[i], q4c[i]])
        rot_g_inv = inv(rot_g)
        delta_rot = rot_c * rot_g_inv
        trace_rot = np.trace( delta_rot )
#         print trace_rot
  
        rot_err[i] = np.arccos ( (trace_rot-1)/2 ) * 180.0 / math.pi;
        gt_index = gt_index+1
    return rot_err

In [394]:
rot_err = rotationError3D(ground_truth_poses[4], # [col][row]
                          ground_truth_poses[5],
                          ground_truth_poses[6],
                          ground_truth_poses[7],
                          corrected_poses[5],
                          corrected_poses[6],
                          corrected_poses[7],
                          corrected_poses[8])

# ground_truth_poses[4].shape[0]
print transl_err, rot_err

[[  2.64879944e-04]
 [  2.84325264e-03]
 [  4.63086450e-03]
 [  7.14660885e-03]
 [  8.05817237e-03]
 [  1.48410348e-02]
 [  2.18859160e-02]
 [  2.82129358e-02]
 [  2.31148477e-01]
 [  3.46253028e-02]
 [  3.63587059e-02]
 [  3.81042814e-02]
 [  2.20003254e-01]
 [  3.84701914e-02]
 [  2.49076153e-01]
 [  3.69123952e-02]
 [  1.53519345e-01]
 [  4.28097491e-02]
 [  4.89965529e-02]
 [  4.76661821e-02]
 [  5.99848424e-02]
 [  2.35065344e-01]
 [  1.60494158e+00]
 [  6.52771413e+04]
 [  4.34034873e+00]
 [  8.92916593e+00]
 [  9.02939551e-01]
 [  6.12755173e-01]
 [  4.22323409e+00]
 [  6.73482132e+00]
 [  9.90905582e+00]
 [  4.31610733e-01]
 [  4.66984144e-01]
 [  4.39655765e-01]
 [  4.02366210e-01]
 [  4.63857370e-01]
 [  4.21926701e-01]
 [  4.35411507e-01]
 [  4.48029572e-01]
 [  2.97477925e-01]
 [  1.35984999e-01]
 [  1.40178276e-01]
 [  3.26035206e-01]
 [  2.99645050e-01]
 [  2.80450888e-01]
 [  3.67345175e-01]
 [  3.78028416e-01]
 [  5.45560850e-01]
 [  3.99767377e-01]
 [  5.09167209e-01]


In [399]:
# Create Labels given ground truth and errors
transl_thresh = 0.30
rot_thresh = 30

def createLabels(transl_err, rot_err):
    
    labels = np.zeros((transl_err.shape[0],1))

    for i in range(0, labels.shape[0]):
        if transl_err[i] > transl_thresh and rot_err[i] > rot_thresh:
            labels[i] = 1.0

    return labels

labels = createLabels(transl_err, rot_err)

print labels

[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]

In [400]:
probs_ours = results[5]
probs_degeneracy = results[6]
probs_icn = results[7]

# ROC Curve and cut-off point
fpr_ours, tpr_ours, threshold_ours = metrics.roc_curve(labels, probs_ours, pos_label=1)
roc_ours_auc = metrics.auc(fpr_ours, tpr_ours)

fpr_degeneracy, tpr_degeneracy, threshold_degeneracy = metrics.roc_curve(labels, probs_degeneracy, pos_label=1)
roc_degeneracy_auc = metrics.auc(fpr_degeneracy, tpr_degeneracy)

fpr_icn, tpr_icn, threshold_icn = metrics.roc_curve(labels, probs_icn, pos_label=1)
roc_icn_auc = metrics.auc(fpr_icn, tpr_icn)

# Plot ROC Curve
plt.title('ROC Curve for Failure Prediction')
plt.plot(fpr_ours, tpr_ours, 'r', linewidth=2.0, label = 'Ours AUC = %0.2f' % roc_ours_auc)
# plt.plot(fpr_degeneracy, tpr_degeneracy, 'b', linewidth=2.0, label = 'Degeneracy AUC = %0.2f' % roc_degeneracy_auc)
# plt.plot(fpr_icn, tpr_icn, 'g', linewidth=2.0, label = 'ICN AUC = %0.2f' % roc_icn_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [401]:
# Precision-Recall Score
average_precision = average_precision_score(labels, probs_ours)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

Average precision-recall score: 0.61


In [402]:
# Plot Precision-Recall Curve

precision, recall, _ = precision_recall_curve(labels, probs_ours, pos_label=1)

plt.step(recall, precision, color='r',
         where='post', linewidth=2.0, label = 'Ours AUC = %0.2f' % average_precision)
plt.legend(loc = 'lower right')
# plt.fill_between(recall, precision, step='post', alpha=0.2,
#                  color='b')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-Recall Curve for Failure Prediction')
plt.show()

In [403]:
risk_threshold = 0.60

# Fill error matrix
def samplemat(dims, labels, probs_ours):
    matrix = np.zeros(dims)
    line_count = 0
    for i in range(min(dims)):
        for j in range(min(dims)):
            print labels[line_count], probs_ours[line_count]
            if (labels[line_count]==1.0 and probs_ours[line_count]>risk_threshold):
                matrix[i, j] = 1.0
            elif (labels[line_count]==0.0 and probs_ours[line_count]<=risk_threshold):
                matrix[i, j] = 0.5
            line_count = line_count + 1
    return matrix

In [404]:
#####################################
# Plot Failure Prediction Map (ours)
#####################################
fig = plt.figure()
ax = fig.add_subplot(111)
cmap = cm.get_cmap('jet')

ax.set_xlabel('Cloud A')
ax.set_ylabel('Cloud B')
ax.xaxis.set_label_position('top')

# Display matrix
mat_size = int(math.sqrt(labels.shape[0]))
# pass octree-overlap column to be taken from results
cax = ax.matshow(samplemat((mat_size, mat_size), labels, probs_ours), cmap=cmap)

# extract all colors
cmaplist = [cmap(i) for i in range(cmap.N)]
# create the new map
cmap = cmap.from_list('pred_cmap', cmaplist, cmap.N)
# define the bins and normalize
bounds = np.linspace(0,1,21)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
# create a second axes for the colorbar
ax2 = fig.add_axes([0.83, 0.1, 0.03, 0.8])
cb = mpl.colorbar.ColorbarBase(ax2, cmap=cmap, norm=norm, spacing='proportional', ticks=bounds, boundaries=bounds)
ax2.set_ylabel('Octree-based Overlap [%]')

plt.show()

[ 0.] [ 0.151991]
[ 0.] [ 0.188626]
[ 0.] [ 0.317506]
[ 0.] [ 0.252902]
[ 0.] [ 0.41375]
[ 0.] [ 0.512209]
[ 0.] [ 0.389623]
[ 0.] [ 0.482656]
[ 0.] [ 0.6215]
[ 0.] [ 0.621448]
[ 0.] [ 0.620161]
[ 0.] [ 0.623614]
[ 0.] [ 0.48162]
[ 0.] [ 0.484565]
[ 0.] [ 0.490381]
[ 0.] [ 0.623897]
[ 0.] [ 0.429949]
[ 0.] [ 0.623381]
[ 0.] [ 0.623143]
[ 0.] [ 0.622988]
[ 0.] [ 0.62306]
[ 0.] [ 0.622998]
[ 1.] [ 0.622976]
[ 1.] [ 0.622827]
[ 1.] [ 0.622776]
[ 1.] [ 0.62281]
[ 1.] [ 0.622985]
[ 1.] [ 0.614461]
[ 1.] [ 0.622701]
[ 1.] [ 0.622676]
[ 1.] [ 0.622675]
[ 0.] [ 0.114635]
[ 0.] [ 0.051]
[ 0.] [ 0.133882]
[ 0.] [ 0.0749125]
[ 0.] [ 0.17832]
[ 0.] [ 0.546591]
[ 0.] [ 0.557926]
[ 0.] [ 0.583136]
[ 0.] [ 0.621621]
[ 0.] [ 0.621397]
[ 0.] [ 0.620193]
[ 1.] [ 0.619762]
[ 0.] [ 0.561917]
[ 0.] [ 0.571901]
[ 1.] [ 0.5493]
[ 0.] [ 0.624033]
[ 0.] [ 0.46935]
[ 0.] [ 0.527133]
[ 0.] [ 0.568475]
[ 0.] [ 0.623129]
[ 0.] [ 0.623102]
[ 0.] [ 0.623036]
[ 1.] [ 0.623045]
[ 1.] [ 0.622876]
[ 1.] [ 0.622831]
[ 1.

[ 0.] [ 0.343489]
[ 0.] [ 0.674315]
[ 0.] [ 0.666586]
[ 1.] [ 0.675072]
[ 1.] [ 0.675155]
[ 1.] [ 0.484677]
[ 1.] [ 0.422394]
[ 1.] [ 0.466785]
[ 0.] [ 0.0588473]
[ 0.] [ 0.086028]
[ 0.] [ 0.0568865]
[ 0.] [ 0.029126]
[ 0.] [ 0.0194878]
[ 0.] [ 0.388945]
[ 0.] [ 0.638311]
[ 0.] [ 0.627505]
[ 0.] [ 0.624254]
[ 0.] [ 0.623614]
[ 1.] [ 0.623902]
[ 1.] [ 0.622475]
[ 1.] [ 0.608644]
[ 1.] [ 0.623011]
[ 1.] [ 0.622885]
[ 1.] [ 0.622755]
[ 1.] [ 0.54831]
[ 1.] [ 0.527045]
[ 0.] [ 0.489582]
[ 0.] [ 0.535431]
[ 0.] [ 0.50331]
[ 0.] [ 0.320645]
[ 0.] [ 0.143517]
[ 0.] [ 0.312453]
[ 0.] [ 0.640165]
[ 1.] [ 0.637171]
[ 1.] [ 0.636815]
[ 1.] [ 0.638256]
[ 1.] [ 0.555549]
[ 1.] [ 0.520319]
[ 1.] [ 0.474556]
[ 0.] [ 0.100602]
[ 0.] [ 0.0340871]
[ 0.] [ 0.0364195]
[ 0.] [ 0.10619]
[ 0.] [ 0.146108]
[ 0.] [ 0.249178]
[ 0.] [ 0.566631]
[ 0.] [ 0.630495]
[ 0.] [ 0.625156]
[ 0.] [ 0.624558]
[ 1.] [ 0.624536]
[ 1.] [ 0.626064]
[ 1.] [ 0.600204]
[ 1.] [ 0.623421]
[ 1.] [ 0.622872]
[ 1.] [ 0.622848]
[ 0.] [ 